# TRABAJO PRÁCTICO INTEGRADOR
## Seminario Procesamiento de lenguaje Natural
### Estudiantes:
#### Romina De León
#### Matías Sarti
#### Magali Wettstein

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Scraping de Noticias

Hacer un scraping de 5 noticias seleccionadas de algunos de los siguientes portales.

a. Infobae.

b. La Política Online.

c. Letra P

d. ADN Ciudad.

e. Portal de noticias a elección (sugerir).

Inicialmente cargué todos las librerías necesarias

In [2]:
#Primera parte. Vamos a instalar la librería que utilizaremos.
!pip install lxml

In [3]:
#Segunda parte, instalamos Beautiful Soup
!pip install BeautifulSoup4

In [4]:
#Por último, instalamos requests
!pip install requests

In [5]:
#Acá sumé el nuevo diccionario para mejorar y amplicar la cantidad de entidades
!pip install -U spacy
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.
2023-07-29 15:33:01.121290: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 15:33:03.437016: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.2

In [6]:
#Una vez que tenemos todas las librerías, vamos a empezar importarlas
import requests
import pandas as pd
from datetime import datetime,date
from bs4 import BeautifulSoup
import pandas as pd
import spacy
import re

In [7]:

import json
import os


In [8]:
from bs4 import BeautifulSoup
import requests



In [9]:
#Para poder utilizarlo, primero vamos a tener que cargarlo en nuestro objeto nlp
nlp = spacy.load('es_core_news_lg')

In [57]:
!mkdir -p "/content/drive/My Drive/tp-integrador-nlp"

In [11]:
#solo tomé la funcion que generó zacarías y la modifiqué un poco para que tome lo que queríamos con las url, sumé la modificación para títulos cuando tenía /n

def extractor_titulos(url, titulo_attrs, diario, filtro_attrs=None): # attrs es un diccionario que contiene atributos y valores para encontrar los títulos dentro del HTML
    # 1° TAREA - EXTRAE
    response = requests.get(url=url)

    # 2° TAREA - TRANSFORMAR
    texto_html = BeautifulSoup(response.text, 'lxml')

    # 3° TAREA - FILTRA
    titulos = texto_html.find_all(**titulo_attrs)

    # 4° TAREA - GENERO UNA LISTA DE NOTICIAS CON EL TEXTO Y LA URL
    lista_titulos = []
    for lista in titulos:
        titulo = lista.text.strip()
        url_noticia = lista.find('a')['href'] if lista.find('a') else None
        if titulo and url_noticia:
            lista_titulos.append({'titulo': titulo, 'url': url_noticia})

    # 5° TAREA - GENERO MI DATAFRAME
    df_titulos = pd.DataFrame(lista_titulos)

    # 6° TAREA - GENERO LA FECHA
    df_titulos['fecha'] = date.today().strftime('%d-%m-%Y')

    # 7° TAREA - GENERO LA LONGITUD DE CARACTERES
    df_titulos['cantidad_caracteres'] = df_titulos['titulo'].apply(lambda x: len(x))

    # 8° TAREA - GENERO EL NOMBRE DEL DIARIO
    df_titulos['diario'] = diario

    # Función para obtener las entidades de un texto utilizando spaCy
    def obtener_entidades(texto):
        doc = nlp(texto)
        return [ent.text for ent in doc.ents if not any(token.is_punct for token in ent)]

    # 10° TAREA - ENTIDADES
    df_titulos['entidades'] = df_titulos['titulo'].apply(obtener_entidades)

    # 9° TAREA - FILTRO
    if filtro_attrs:
        for attr, value in filtro_attrs.items():
            df_titulos = df_titulos[df_titulos[attr] > value]

    return df_titulos


In [12]:
#acá generé el df como hizo zacarías, agregué la diferencia de atributos de div en el título
df_noticias_ambito = extractor_titulos(url='https://www.ambito.com/',
                                        titulo_attrs={'name': 'h2'},
                                        diario='Ámbito')

In [13]:
df_noticias_ambito.head(15)

,titulo,url,fecha,cantidad_caracteres,diario,entidades
0,Triple Frontera: Aduana confiscó anabólicos va...,https://www.ambito.com/economia/triple-fronter...,29-07-2023,74,Ámbito,"[Triple Frontera, Aduana]"
1,Creció la compra de dólar ahorro en junio: alc...,https://www.ambito.com/los-argentinos-compraro...,29-07-2023,70,Ámbito,[]
2,Detienen a hijo del presidente de Colombia en ...,https://www.ambito.com/mundo/detienen-hijo-del...,29-07-2023,99,Ámbito,[de Colombia]
3,Línea Roca con servicio limitado: desde cuándo...,https://www.ambito.com/informacion-general/lin...,29-07-2023,69,Ámbito,[Línea Roca]
4,El FMI insiste en la acumulación de reservas y...,https://www.ambito.com/economia/el-fmi-insiste...,29-07-2023,70,Ámbito,[FMI]
5,La Aduana pone la lupa sobre la industria del ...,https://www.ambito.com/economia/la-aduana-pone...,29-07-2023,82,Ámbito,[Aduana]
6,Plazo fijo: ¿cuánto gano hoy si invierto $100....,https://www.ambito.com/finanzas/plazo-fijo-cua...,29-07-2023,50,Ámbito,[]
7,Clave para el dólar: seguirán el crawling peg ...,https://www.ambito.com/clave-el-dolar-seguira-...,29-07-2023,91,Ámbito,[]
8,Sergio Massa extiende su presencia en las prov...,https://www.ambito.com/politica/sergio-massa-e...,29-07-2023,83,Ámbito,"[Sergio Massa, Tucumán]"
9,"Aumentos de agosto: uno por uno, los 8 aumento...",https://www.ambito.com/economia/aumentos-agost...,29-07-2023,61,Ámbito,[]


In [14]:
df_noticias_clarin = extractor_titulos(url='https://www.clarin.com',
                  titulo_attrs={'name': 'h2'},
                  diario='Clarin')

In [15]:
df_noticias_clarin
##pero trae otras cosas

,titulo,url,fecha,cantidad_caracteres,diario,entidades
0,Dólar blue,https://www.clarin.com/economia/dolar-blue-hoy...,29-07-2023,10,Clarin,[Dólar blue]
1,Dónde voto,https://www.clarin.com/politica/voto-consulta-...,29-07-2023,10,Clarin,[]
2,Elecciones 2023,https://www.clarin.com/politica/elecciones-202...,29-07-2023,15,Clarin,[]
3,Sergio Massa,https://www.clarin.com/politica/miedo-pasarla-...,29-07-2023,12,Clarin,[Sergio Massa]
4,Patricia Bullrich,https://www.clarin.com/politica/-campora-paso-...,29-07-2023,17,Clarin,[Patricia Bullrich]
5,FMI,https://www.clarin.com/economia/hace-50-anos-f...,29-07-2023,3,Clarin,[FMI]
6,Conventos en clausura,https://www.clarin.com/sociedad/conventos-cla...,29-07-2023,21,Clarin,[]
7,Tragedia en el aire,https://www.clarin.com/deportes/sudafrica-vs-a...,29-07-2023,19,Clarin,[Tragedia]
8,Los Pumas vs Sudáfrica,https://www.clarin.com/deportes/sudafrica-vs-p...,29-07-2023,22,Clarin,"[Pumas, Sudáfrica]"
9,Mundial Femenino,https://www.clarin.com/deportes/mundial-femeni...,29-07-2023,16,Clarin,[Mundial Femenino]


Ahora deberíamos trabjar con las url y descargar las noticias, o se les ocurre otra forma?
Dejo abajo copiado lo de la clase de como tratar los parrafos

In [16]:
df_ambito = df_noticias_ambito[df_noticias_ambito["entidades"].apply(lambda x: "FMI" in x)]

df_ambito

,titulo,url,fecha,cantidad_caracteres,diario,entidades
4,El FMI insiste en la acumulación de reservas y...,https://www.ambito.com/economia/el-fmi-insiste...,29-07-2023,70,Ámbito,[FMI]
10,Pago al FMI: la CAF autorizó un préstamo puent...,https://www.ambito.com/economia/pago-al-fmi-la...,29-07-2023,83,Ámbito,"[FMI, CAF, u$s1.000, la Argentina]"
18,Cómo cayó en los mercados el acuerdo con el FMI,https://www.ambito.com/opiniones/como-cayo-los...,29-07-2023,47,Ámbito,[FMI]
25,Mercado celebró acuerdo con FMI: cuáles fueron...,https://www.ambito.com/finanzas/mercado-celebr...,29-07-2023,103,Ámbito,[FMI]
55,"Massa: ""El acuerdo con el FMI nos da tranquili...",https://www.ambito.com/economia/sergio-massa-e...,29-07-2023,89,Ámbito,"[Massa, FMI]"
83,Argentina llegó a un acuerdo con el FMI y reci...,https://www.ambito.com/economia/confirmado-arg...,29-07-2023,69,Ámbito,"[Argentina, FMI]"
113,Las claves del acuerdo entre la Argentina y el...,https://www.ambito.com/economia/las-claves-del...,29-07-2023,50,Ámbito,"[la Argentina, FMI]"
116,Acuerdo con el FMI: plan de contención de sala...,https://www.ambito.com/acuerdo-fmi-plan-conten...,29-07-2023,77,Ámbito,[FMI]
117,Acuerdo con el FMI: las repercusiones en medio...,https://www.ambito.com/economia/acuerdo-el-fmi...,29-07-2023,63,Ámbito,[FMI]
119,Acuerdo con el FMI: cómo quedó la nueva meta d...,https://www.ambito.com/economia/acuerdo-el-fmi...,29-07-2023,56,Ámbito,[FMI]


In [17]:
#Ahora deberíamos ver de entrar a las url de cada noticia y empezar el análisis
#requests.get(url)

In [18]:
df_seleccion = df_ambito.head(5)[["url"]].copy()
df_seleccion

,url
4,https://www.ambito.com/economia/el-fmi-insiste...
10,https://www.ambito.com/economia/pago-al-fmi-la...
18,https://www.ambito.com/opiniones/como-cayo-los...
25,https://www.ambito.com/finanzas/mercado-celebr...
55,https://www.ambito.com/economia/sergio-massa-e...


Esta función toma una URL como entrada y realiza el scraping de la página correspondiente. Primero, hace una solicitud HTTP GET a la URL para obtener el contenido de la página. Luego, crea un objeto BeautifulSoup para analizar el contenido HTML de la página.

El código busca el título de la noticia dentro de la etiqueta 'h1' y lo extrae. Luego, busca todos los párrafos 'p' en la página y filtra aquellos párrafos que contienen más de 10 palabras se considera un párrafo relevante. Finalmente, devuelve el título y la lista de párrafos seleccionados.

In [19]:

# Función para realizar el scraping de una noticia y extraer sus párrafos y titulos
def scrape_noticia(url):
    response = requests.get(url) #Realizar una solicitud GET a la URL para obtener el contenido de la página web
    for index, fila in df_ambito.iterrows(): #acá agregué para que lo haga dentro de nuestro df_ambito
        url = fila['url']
    soup = BeautifulSoup(response.content, 'html.parser') # # Crear un objeto BeautifulSoup para analizar el contenido HTML de la página
    title = soup.find('h1').get_text().strip() # Encontrar el título de la noticia en la etiqueta <h1> y extraer su texto
    paragraphs = soup.find_all('p') ## Encontrar todos los párrafos (<p>) en la página y filtrar aquellos con más de 10 palabras
    selected_paragraphs = [p.get_text().strip() for p in paragraphs if len(p.get_text().split()) > 10]
    return title, selected_paragraphs  # Devolver el título y la lista de párrafos seleccionados




En este bloque de código, iteramos sobre las URLs de las noticias y llamamos a la función scrape_noticia para obtener el título y los párrafos relevantes de cada noticia. Luego, almacenamos esta información en una lista llamada corpus, que luego se utilizará para construir el DataFrame.

Dentro del DataFrame, guardamos el título, los párrafos, la cantidad de párrafos, la cantidad de palabras en los párrafos y una lista vacía para las entidades nombradas. Debes implementar la lógica para extraer las entidades nombradas según tus requisitos.

In [20]:
# Lista para almacenar los párrafos de todas las noticias

corpus = []

for url in df_seleccion["url"]: #acá modifiqué el cod de magui
    title, paragraphs = scrape_noticia(url)
    entities_list = [token.text for entity in nlp(title).ents for token in entity if not token.is_punct] #acá deberíamos ver porque la lista de entidades no da bien, podríamos usar la función definida más arriba para obtener_entidades
    corpus.append({
        'Título': title,
        'Párrafos': paragraphs,
        'Cantidad de párrafos': len(paragraphs),
        'Cantidad de palabras': sum(len(p.split()) for p in paragraphs),
        'Lista de entidades': entities_list #ver tema entidades que pone como lista son medio cualca
    })

df = pd.DataFrame(corpus)


In [21]:
df

,Título,Párrafos,Cantidad de párrafos,Cantidad de palabras,Lista de entidades
0,El FMI insiste en la acumulación de reservas y...,[Negociación. Sergio Massa y el equipo económi...,11,546,[FMI]
1,Pago al FMI: la CAF autorizó un préstamo puent...,[A través de una reunión extraordinaria de Dir...,6,235,"[FMI, CAF, u$s1.000, la, Argentina]"
2,Cómo cayó en los mercados el acuerdo con el FMI,[Finalmente se conoció el acuerdo que logró el...,19,845,[FMI]
3,Mercado celebró acuerdo con FMI: cuáles fueron...,[Mercado celebra acuerdo con el FMI: acciones ...,30,1334,[FMI]
4,"Sergio Massa: ""El acuerdo con el FMI nos da tr...","[poSergio Massa, Edaurdo ""Wado"" De Pedro y Ser...",8,537,"[Sergio, Massa, FMI]"


# Podemos probar normalizando el texto porque seguro no reconoce los acentos


In [22]:
!pip install unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 2.1 MB/s eta 0:00:00


In [23]:
from unidecode import unidecode

In [24]:
def lematizar_parrafo(parrafo):
        doc = nlp(parrafo)
        lemas = [token.lemma_ for token in doc if not token.is_punct] #eliminé las puntuaciones
        return ' '.join(lemas)

In [25]:
# Lista para almacenar los párrafos de todas las noticias

corpus = []

for url in df_seleccion["url"]: #acá modifiqué el cod de magui
    title, paragraphs = scrape_noticia(url)
    normalized_title = unidecode(title)
    entities_list = [token.text for entity in nlp(normalized_title).ents for token in entity if not token.is_punct] #acá deberíamos ver porque la lista de entidades no da bien, podríamos usar la función definida más arriba para obtener_entidades
    lematized_paragraphs = [lematizar_parrafo(paragraph) for paragraph in paragraphs]

    corpus.append({
        'Título': normalized_title,
        'Párrafos': paragraphs,
        'Cantidad de párrafos': len(paragraphs),
        'Cantidad de palabras': sum(len(p.split()) for p in paragraphs),
        'Lista de entidades': entities_list, #ver tema entidades que pone como lista son medio cualca
        'Párrafos lematizados': lematized_paragraphs
    })

df = pd.DataFrame(corpus)

In [26]:
df

,Título,Párrafos,Cantidad de párrafos,Cantidad de palabras,Lista de entidades,Párrafos lematizados
0,El FMI insiste en la acumulacion de reservas y...,[Negociación. Sergio Massa y el equipo económi...,11,546,[FMI],[Negociación Sergio Massa y el equipo económic...
1,Pago al FMI: la CAF autorizo un prestamo puent...,[A través de una reunión extraordinaria de Dir...,6,235,"[FMI, CAF, u$s1.000, la, Argentina]",[a través de uno reunión extraordinario de Dir...
2,Como cayo en los mercados el acuerdo con el FMI,[Finalmente se conoció el acuerdo que logró el...,19,845,[FMI],[finalmente él conocer el acuerdo que lograr e...
3,Mercado celebro acuerdo con FMI: cuales fueron...,[Mercado celebra acuerdo con el FMI: acciones ...,30,1334,[FMI],[Mercado celebrar acuerdo con el FMI acción y ...
4,"Sergio Massa: ""El acuerdo con el FMI nos da tr...","[poSergio Massa, Edaurdo ""Wado"" De Pedro y Ser...",8,537,"[Sergio, Massa, FMI]",[posergio Massa Edaurdo Wado De Pedro y Sergio...


In [62]:
df.to_excel('/content/drive/My Drive/tp-integrador-nlp/noticias.xlsx', index=False)



In [63]:
import os

# df = ...

# Función para guardar las columnas 'Título', 'Párrafos' y 'Lista de entidades' de una fila como archivo de texto
def save_row_as_text(index):
    try:
        row_data = df.loc[index, ['Título', 'Lista de entidades', 'Párrafos']].to_dict()
        file_path = os.path.join('/content/drive/My Drive/tp-integrador-nlp', f'row_{index}.txt')
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(f"Título: {row_data['Título']}\n")
            file.write(f"Lista de entidades: {row_data['Lista de entidades']}\n")
            file.write("Párrafos:\n")
            for paragraph in row_data['Párrafos']:
                file.write(paragraph + "\n")
    except IndexError:
        print(f"La fila con índice {index} no existe.")

# Guardar las columnas 'Título', 'Párrafos' y 'Lista de entidades' de cada fila como archivo de texto
for index in range(len(df)):
    save_row_as_text(index)

#PROBANDO

In [61]:
df.head()

,Título,Párrafos,Cantidad de párrafos,Cantidad de palabras,Lista de entidades,Párrafos lematizados
0,El FMI insiste en la acumulacion de reservas y...,[Negociación. Sergio Massa y el equipo económi...,11,546,[FMI],[Negociación Sergio Massa y el equipo económic...
1,Pago al FMI: la CAF autorizo un prestamo puent...,[A través de una reunión extraordinaria de Dir...,6,235,"[FMI, CAF, u$s1.000, la, Argentina]",[a través de uno reunión extraordinario de Dir...
2,Como cayo en los mercados el acuerdo con el FMI,[Finalmente se conoció el acuerdo que logró el...,19,845,[FMI],[finalmente él conocer el acuerdo que lograr e...
3,Mercado celebro acuerdo con FMI: cuales fueron...,[Mercado celebra acuerdo con el FMI: acciones ...,30,1334,[FMI],[Mercado celebrar acuerdo con el FMI acción y ...
4,"Sergio Massa: ""El acuerdo con el FMI nos da tr...","[poSergio Massa, Edaurdo ""Wado"" De Pedro y Ser...",8,537,"[Sergio, Massa, FMI]",[posergio Massa Edaurdo Wado De Pedro y Sergio...


In [46]:
df[['Párrafos lematizados']]

,Párrafos lematizados
0,[Negociación Sergio Massa y el equipo económic...
1,[a través de uno reunión extraordinario de Dir...
2,[finalmente él conocer el acuerdo que lograr e...
3,[Mercado celebrar acuerdo con el FMI acción y ...
4,[posergio Massa Edaurdo Wado De Pedro y Sergio...


In [50]:
!pip install nbconvert



In [77]:
current_path = os.getcwd()
current_path


'/content'

In [78]:
#Si quiero comprimir la carpeta con los archivos
import shutil
shutil.make_archive('tp-integrador-nlp', 'zip', '/content/drive/My Drive/tp-integrador-nlp/')

'/content/tp-integrador-nlp.zip'

In [79]:
#Si quiero descargar la carpeta comprimida desde colab
from google.colab import files
files.download(f'tp-integrador-nlp.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>